# Data Preparation
This notebook prepares the survey data by splitting the data into the three regions. This notebook includes selecting the base map for plotting, followed by subsetting the survey data in the three different regions. Before this takes place, I need to import the appropiate packages, read the relevant csv files, rename columns and define functions. This is the first notebook that needs to be run before using the other notebooks.

## 0. Data preperation before splitting

### 0.1 Import packages

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np

### 0.2 Reading csv files
For splitting the data, an extrenal MSOA dataset is used from the House of Commons under the Open Parliament Licence v3.0.

In [2]:
#Reading the MSOA dataset from the House of Commons
msoa_all= pd.read_csv('../datafiles/all_msoa.csv')
#Reading and loading in the different csv files with pandas
survey = pd.read_csv('../datafiles/SCALAR_Coastal_Study_new_respondents_Wave_Five_UK.csv') 
#This is the data containing the names of the corresponding MSOA Codes
names = pd.read_excel('../datafiles/msoa_names.xlsx')

In [3]:
#Showing the msoa_all dataframe
msoa_all.head()

,msoa11cd,msoa11nm,msoa11nmw,msoa11hclnm,msoa11hclnmw,Laname
0,E02000001,City of London 001,City of London 001,City of London,NaN,City of London
1,E02000002,Barking and Dagenham 001,Barking and Dagenham 001,Marks Gate,NaN,Barking and Dagenham
2,E02000003,Barking and Dagenham 002,Barking and Dagenham 002,Chadwell Heath East,NaN,Barking and Dagenham
3,E02000004,Barking and Dagenham 003,Barking and Dagenham 003,Eastbrookend,NaN,Barking and Dagenham
4,E02000005,Barking and Dagenham 004,Barking and Dagenham 004,Becontree Heath,NaN,Barking and Dagenham


In [4]:
#Showing the survey_data dataframe
survey.head()

,ID,Q0_place_UK,Q1_home_NL_UK,Q4_home_size_UK,Q5_home_tenure,Q5b_home_sell,Q6_home_costs,Q7_move_in,Q8_move_out,Q12_neighborhood_trust,...,Q29h_com_what_other_p2,Q29h_com_what_other_p3,Q35_responsibility_citizens,Q35_responsibility_business,Q35_responsibility_ngo_local,Q35_responsibility_ngo_international,Q35_responsibility_gov_local,Q35_responsibility_gov_national,Q35_responsibility_w_authorities,Q35_responsibility_other
0,0,5456,4,2,2,350000,300,2015,4,3,...,NaN,NaN,0,0,0,0,0,100,0,0
1,1,5451,3,2,2,350000,3000,1998,4,4,...,NaN,NaN,0,0,0,0,50,0,50,0
2,2,6170,3,5,2,265000,300,2012,98,5,...,NaN,NaN,10,0,0,10,50,0,30,0
3,3,6101,3,6,2,550000,3000,2019,3,3,...,NaN,NaN,0,0,0,0,100,0,0,0
4,4,6160,4,6,2,385000,280,2000,2,4,...,NaN,NaN,0,0,0,0,30,35,35,0


In [5]:
#Showing the names dataframe
names.head()

,RecordNo,msoa11
0,0,North Norfolk 012
1,1,North Norfolk 007
2,2,Waveney 006
3,3,Babergh 011
4,4,Suffolk Coastal 011


###  0.3 Renaming columns for further analysis

In [6]:
#Renaming columns for merging
msoa_all=msoa_all.rename(columns={'msoa11nm' : 'MSOA name', 'Laname': 'LA name'})
names=names.rename(columns={'RecordNo': 'ID', 'msoa11': 'MSOA name' })

In [7]:
names

,ID,MSOA name
0,0,North Norfolk 012
1,1,North Norfolk 007
2,2,Waveney 006
3,3,Babergh 011
4,4,Suffolk Coastal 011
...,...,...
738,746,Tower Hamlets 004
739,747,Wandsworth 017
740,748,Enfield 010
741,749,Tower Hamlets 013


### 0.4 Functions
In this section, I created a function to split the dataset up per region.

In [8]:
def find_regions(df, start_string, end_string):
    #Define which entry is the start of the dataframe by inputting a string
    start_index_df = df.index.get_loc(df[df['msoa11nmw'] == start_string].index[0])
    #Define the end of the dataframe by inputting a string
    end_index_df = df.index.get_loc(df[df['msoa11nmw'] == end_string].index[-1])
    #Create a new dataframe from the start string till the end string + 1 extra entry to include the end string
    new_df=df.iloc[start_index_df:end_index_df + 1]
    #Returns the new dataframe
    return new_df

## 1. Merging original survey data frame with msoa names 
For merging with the geodataframe afterwards, I need to merge the survey dataframe with a names dataframe that contains the names of the MSOA codes. This is needed as the MSOA codes in the survey dataframe and the MSOA codes in the shapefile are different, hence I will merge on the area names of the MSOA codes for further analysis in this notebook.

In [9]:
#Merging the two dataframes 
survey_data=survey.merge(names, on='ID',  how="left")

In [10]:
#This is the new merged dataframe that will be used for further analysis 
survey_data.head()

,ID,Q0_place_UK,Q1_home_NL_UK,Q4_home_size_UK,Q5_home_tenure,Q5b_home_sell,Q6_home_costs,Q7_move_in,Q8_move_out,Q12_neighborhood_trust,...,Q29h_com_what_other_p3,Q35_responsibility_citizens,Q35_responsibility_business,Q35_responsibility_ngo_local,Q35_responsibility_ngo_international,Q35_responsibility_gov_local,Q35_responsibility_gov_national,Q35_responsibility_w_authorities,Q35_responsibility_other,MSOA name
0,0,5456,4,2,2,350000,300,2015,4,3,...,NaN,0,0,0,0,0,100,0,0,North Norfolk 012
1,1,5451,3,2,2,350000,3000,1998,4,4,...,NaN,0,0,0,0,50,0,50,0,North Norfolk 007
2,2,6170,3,5,2,265000,300,2012,98,5,...,NaN,10,0,0,10,50,0,30,0,Waveney 006
3,3,6101,3,6,2,550000,3000,2019,3,3,...,NaN,0,0,0,0,100,0,0,0,Babergh 011
4,4,6160,4,6,2,385000,280,2000,2,4,...,NaN,0,0,0,0,30,35,35,0,Suffolk Coastal 011


In [11]:
#Writing the new dataframe to a csv file for further use. This file, without transformation, will be only used for initial exploration in Step0Exploratory
survey_data.to_csv('../datafiles/merged_names.csv', index=False)

## 2. Transforming and Renaming Columns 
Before I write this new dataframe with the MSOA names to a csv file, I want to transform one column. 'Q0_age' is a a ratio type of variable, but for my clustering I want this to be a Likert scale variable. Hence, I transform this variable in this notebook for easier implementation later on. 

In [12]:
#These are the bins and labels for the age group
bins = [18, 30, 45, 60, 75, 92]
labels =[1, 2, 3, 4 , 5]

In [13]:
#Creating a new columns based on the bins and labels
survey_data['Age Group'] = pd.cut(survey_data['Q0_age'], bins=bins, labels=labels, right=False)
#Dropping the old column
survey_data.drop('Q0_age', axis=1, inplace= True)

In [14]:
#Renaming columns
survey_data = survey_data.rename(columns={'R02_perc_prob': 'Perceived Flood Probability', 'R05_worry' : 'Worry Flood', 'Q18_flood_exp' : 'Flood Experience', 
                                           'R03_perc_damage' : 'Perceived Physical Damage', 'Q44_trust_pm' : 'Trust Prime Minister', 
                                           'Q44_trust_gov_rep' : 'Trust Government Representative', 'Q44_trust_family': 'Trust Family and Friends', 
                                           'Q44_trust_media' : 'Trust General Media', 'Q44_trust_s_media': 'Trust Social Media','Q53_income_UK': 'Household Income', 
                                          'Q58_savings': 'Household Savings', 'Q51_unempl_time': 'Unemployment Time' ,'Q52_multiple_income': 'Multiple Incomes' })


In [15]:
#Writing the new dataframe to a csv file for further use. This is the file that will be used for further, such as in Step0GroupMeasures
#and in Step1Descriptives
survey_data.to_csv('../datafiles/transformed_names.csv', index=False)

## 3. Finding the Regions
In this section, subsetting of the regions will be done according to the MSOA names

### Norfolk 
Selecting all the local authority names belonging to Norfolk. This is done manually by searching on Google which MSOA regions belong to Norfolk and looking in the csv file where they are located. 

In [16]:
start_norfolk = 'Breckland 001'
end_norfolk = 'South Norfolk 015'
start_norfolk_2='Norwich 014'
end_norfolk_2 = 'Norwich 015'

In [17]:
norfolk = find_regions(msoa_all, start_norfolk, end_norfolk)

In [18]:
norfolk2 = find_regions(msoa_all, start_norfolk_2, end_norfolk_2)

### Suffolk 

Selecting all the local authority names belonging to Suffolk.

In [19]:
start_suffolk = 'Babergh 001'
end_suffolk = 'Waveney 015'

In [20]:
suffolk = find_regions(msoa_all, start_suffolk, end_suffolk)

In [21]:
#Concatenating all Norfolk and Suffolk regions
combined_ns = pd.concat([norfolk, norfolk2, suffolk], ignore_index=True)

In [22]:
#Saving the MSOA names for the region to plot as a base region map for later
combined_ns.to_csv('../datafiles/norfolk_suffolk_region.csv', index=False)

### Somerset

In [23]:
start_somerset1 = 'Mendip 001'
end_somerset1 = 'West Somerset 005'

In [24]:
somerset_1 = find_regions(msoa_all, start_somerset1, end_somerset1)

In [25]:
somerset_2 = msoa_all[msoa_all['LA name'] == 'Bath and North East Somerset']

In [26]:
somerset_3 = msoa_all[msoa_all['LA name'] == 'North Somerset']

In [27]:
#Concatenating all Somerset regions
combined_somerset = pd.concat([somerset_1,somerset_2,somerset_3], ignore_index=True)

In [28]:
#Saving the MSOA names for the region to plot as a base region map for later
combined_somerset.to_csv('../datafiles/somerset_region.csv', index=False)

### Greater London 
London had a lot of scattered regions in the csv file. Hence, a lot of different dataframes had to be concatenated.

In [29]:
start_london1 = 'City of London 001'
end_london1 = 'Westminster 024'
start_london2 = 'Bromley 040'
end_london2 = 'Bromley 042'
start_london3= 'Ealing 040'
end_london3= 'Southwark 034'
start_london4= 'Greenwich 034'
end_london4='Greenwich 038'

In [30]:
extra_london=['Hackney 028', 'Hackney 029', 'Redbridge 035', 'Redbridge 036', 'Harrow 033', 'Tower Hamlets 032', 'Tower Hamlets 033']

In [31]:
london_1 = find_regions(msoa_all, start_london1, end_london1)

In [32]:
london_2 = find_regions(msoa_all, start_london2, end_london2)

In [33]:
london_3=find_regions(msoa_all, start_london3, end_london3)

In [34]:
london_4=find_regions(msoa_all, start_london4, end_london4)

In [35]:
london_5= msoa_all[msoa_all['msoa11nmw'].isin(extra_london)]

In [36]:
#Concatenating all London regions
combined_london = pd.concat([london_1,london_2, london_3, london_4, london_5], ignore_index=True)

In [37]:
#Saving the MSOA names for the region to plot as a base region map for later
combined_london.to_csv('../datafiles/london_region.csv', index=False)

## 4. Filtering survey data dataframe for the three seperate regions
In this section, the survey data is split up according to the three regions and saved into different csv files for further use.

In [38]:
#Selecting the entries in survey data that are in the norfolk/suffolk region. 
filtered_ns=survey_data[survey_data['MSOA name'].isin(combined_ns['MSOA name'])]
filtered_ns.to_csv('../datafiles/filtered_ns.csv', index=False)

In [39]:
#Selecting the entries in survey data that are in the Somerset. 
filtered_somerset=survey_data[survey_data['MSOA name'].isin(combined_somerset['MSOA name'])]
filtered_somerset.to_csv('../datafiles/filtered_somerset.csv', index=False)

In [40]:
#Selecting the entries in survey data that are in the London. 
filtered_london=survey_data[survey_data['MSOA name'].isin(combined_london['MSOA name'])]
filtered_london.to_csv('../datafiles/filtered_london.csv', index=False)